In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [3]:
# initialize policy
policy = np.ones([env.nS, env.nA]) / env.nA

In [4]:
# 16 states with 4 actions per state, action probability is 1 for one out of 4 actions in every state
# implying in every state, there is only one specific action that will be chosen

V = np.zeros(env.nS)
discount_factor=1.0
theta=0.00001

while True:
    delta = 0
    # For each state, perform a "full backup"
    for s in range(env.nS): # there are 16 states
        v = 0
        # Look at the possible next actions
        for a, action_prob in enumerate(policy[s]):
            # For each action, look at the possible next states...
            for  prob, next_state, reward, done in env.P[s][a]:
                # Calculate the expected value
                v += action_prob * prob * (reward + discount_factor * V[next_state])
        # How much our value function changed (across any states)
        delta = max(delta, np.abs(v - V[s]))
        V[s] = v
    # Stop evaluating once our value function change is below a threshold
    if delta < theta:
        break

In [ ]:
V

In [5]:
def one_step_lookahead(state, V):
    """
    Helper function to calculate the value for all action in a given state.

    Args:
        state: The state to consider (int)
        V: The value to use as an estimator, Vector of length env.nS

    Returns:
        A vector of length env.nA containing the expected value of each action.
    """
    A = np.zeros(env.nA)
    for a in range(env.nA):
        for prob, next_state, reward, done in env.P[state][a]:
            A[a] += prob * (reward + discount_factor * V[next_state])
    return A

In [ ]:
action_values

In [6]:
for s in range(env.nS):

    # The best action we would take under the current policy
    chosen_a = np.argmax(policy[s])
    print("states: ", s)
    print("policy: ", policy[s])
    print("chosen a: ", chosen_a)

    # Find the best action by one-step lookahead
    # Ties are resolved arbitarily
    action_values = one_step_lookahead(s, V)
    print("action values: ", action_values)
    best_a = np.argmax(action_values)
    print("best a: ", best_a)

    # Greedily update the policy
    if chosen_a != best_a:
        policy_stable = False
    print("before policy: ", policy[s])
    policy[s] = np.eye(env.nA)[best_a]
    print("after policy: ", policy[s])
    print()

states:  0
policy:  [0.25 0.25 0.25 0.25]
chosen a:  0
action values:  [0. 0. 0. 0.]
best a:  0
before policy:  [0.25 0.25 0.25 0.25]
after policy:  [1. 0. 0. 0.]

states:  1
policy:  [0.25 0.25 0.25 0.25]
chosen a:  0
action values:  [-14.99993529 -20.99990698 -18.9999206   -1.        ]
best a:  3
before policy:  [0.25 0.25 0.25 0.25]
after policy:  [0. 0. 0. 1.]

states:  2
policy:  [0.25 0.25 0.25 0.25]
chosen a:  0
action values:  [-20.99990698 -22.99989761 -20.99991379 -14.99993529]
best a:  3
before policy:  [0.25 0.25 0.25 0.25]
after policy:  [0. 0. 0. 1.]

states:  3
policy:  [0.25 0.25 0.25 0.25]
chosen a:  0
action values:  [-22.99989761 -22.99989761 -20.99991477 -20.99990698]
best a:  3
before policy:  [0.25 0.25 0.25 0.25]
after policy:  [0. 0. 0. 1.]

states:  4
policy:  [0.25 0.25 0.25 0.25]
chosen a:  0
action values:  [ -1.         -18.9999206  -20.99990698 -14.99993529]
best a:  0
before policy:  [0.25 0.25 0.25 0.25]
after policy:  [1. 0. 0. 0.]

states:  5
policy:  

In [ ]:
# # Start with a random policy
# policy = np.ones([env.nS, env.nA]) / env.nA

while True:
    # Evaluate the current policy
    V = policy_eval_fn(policy, env, discount_factor)

    # Will be set to false if we make any changes to the policy
    policy_stable = True

    # For each state...
    for s in range(env.nS):
        # The best action we would take under the current policy
        chosen_a = np.argmax(policy[s])

        # Find the best action by one-step lookahead
        # Ties are resolved arbitarily
        action_values = one_step_lookahead(s, V)
        best_a = np.argmax(action_values)

        # Greedily update the policy
        if chosen_a != best_a:
            policy_stable = False
        policy[s] = np.eye(env.nA)[best_a]

    # If the policy is stable we've found an optimal policy. Return it
    if policy_stable:
        return policy, V

In [7]:
# a 1x4 array of probabilities for each action
policy[0]

array([1., 0., 0., 0.])

In [13]:
policy[0] = np.eye(4)[3]

In [10]:
best_a

0

In [14]:
policy[0]

array([0., 0., 0., 1.])

In [ ]:
# # Taken from Policy Evaluation Exercise!

# def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
#     """
#     Evaluate a policy given an environment and a full description of the environment's dynamics.
    
#     Args:
#         policy: [S, A] shaped matrix representing the policy.
#         env: OpenAI env. env.P represents the transition probabilities of the environment.
#             env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
#             env.nS is a number of states in the environment. 
#             env.nA is a number of actions in the environment.
#         theta: We stop evaluation once our value function change is less than theta for all states.
#         discount_factor: Gamma discount factor.
    
#     Returns:
#         Vector of length env.nS representing the value function.
#     """
#     # Start with a random (all 0) value function
#     V = np.zeros(env.nS)
#     while True:
#         delta = 0
#         # For each state, perform a "full backup"
#         for s in range(env.nS):
#             v = 0
#             # Look at the possible next actions
#             for a, action_prob in enumerate(policy[s]):
#                 # For each action, look at the possible next states...
#                 for  prob, next_state, reward, done in env.P[s][a]:
#                     # Calculate the expected value
#                     v += action_prob * prob * (reward + discount_factor * V[next_state])
#             # How much our value function changed (across any states)
#             delta = max(delta, np.abs(v - V[s]))
#             V[s] = v
#         # Stop evaluating once our value function change is below a threshold
#         if delta < theta:
#             break
#     return np.array(V)

In [ ]:
# def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
#     """
#     Policy Improvement Algorithm. Iteratively evaluates and improves a policy
#     until an optimal policy is found.
    
#     Args:
#         env: The OpenAI environment.
#         policy_eval_fn: Policy Evaluation function that takes 3 arguments:
#             policy, env, discount_factor.
#         discount_factor: gamma discount factor.
        
#     Returns:
#         A tuple (policy, V). 
#         policy is the optimal policy, a matrix of shape [S, A] where each state s
#         contains a valid probability distribution over actions.
#         V is the value function for the optimal policy.
        
#     """

#     def one_step_lookahead(state, V):
#         """
#         Helper function to calculate the value for all action in a given state.
        
#         Args:
#             state: The state to consider (int)
#             V: The value to use as an estimator, Vector of length env.nS
        
#         Returns:
#             A vector of length env.nA containing the expected value of each action.
#         """
#         A = np.zeros(env.nA)
#         for a in range(env.nA):
#             for prob, next_state, reward, done in env.P[state][a]:
#                 A[a] += prob * (reward + discount_factor * V[next_state])
#         return A
    
#     # Start with a random policy
#     policy = np.ones([env.nS, env.nA]) / env.nA
    
#     while True:
#         # Evaluate the current policy
#         V = policy_eval_fn(policy, env, discount_factor)
        
#         # Will be set to false if we make any changes to the policy
#         policy_stable = True
        
#         # For each state...
#         for s in range(env.nS):
#             # The best action we would take under the current policy
#             chosen_a = np.argmax(policy[s])
            
#             # Find the best action by one-step lookahead
#             # Ties are resolved arbitarily
#             action_values = one_step_lookahead(s, V)
#             best_a = np.argmax(action_values)
            
#             # Greedily update the policy
#             if chosen_a != best_a:
#                 policy_stable = False
#             policy[s] = np.eye(env.nA)[best_a]
        
#         # If the policy is stable we've found an optimal policy. Return it
#         if policy_stable:
#             return policy, V

In [ ]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



In [ ]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)